## Tensorflow Scratch Implementation

In [1]:
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
## Generate X, y

### Init Variables
true_w = tf.constant([3, -2.2])
true_b = 4.2
num_samples = 1000

### Generate Samples
features = tf.random.normal((num_samples, true_w.shape[0]), dtype=tf.float32)
labels = tf.matmul(features, tf.reshape(true_w, (-1, 1))) + true_b
labels += tf.random.normal(shape=labels.shape, stddev=0.01)
labels = tf.reshape(labels, (-1, 1))

In [3]:
## Read Batches of data
def data_iter(X, y, batch_size):
    '''Yields batches of X, y'''
    num_samples = len(y)
    indices = list(range(num_samples))
    random.shuffle(indices)

    for i in range(0, num_samples, batch_size):
        j = tf.constant(indices[i: min(i + batch_size, num_samples)])
        yield tf.gather(X, j), tf.gather(y, j)

# next(data_iter(X, y, 32))

In [4]:
## Functions for modeling

def init_params():
    """ Initialize w, b"""
    w = tf.Variable(tf.random.normal(shape = (2, 1), mean = 0, stddev= 0.1), trainable = True)
    b = tf.Variable(tf.zeros(1), trainable = True)
    return w, b

def lin_reg(X, w, b):
    """ Linear Regression : y = wx + b """
    return tf.matmul(X, w) + b

def squared_loss(yhat, y):
    return (yhat - tf.reshape(y, yhat.shape))**2/2

def sgd(params, grads, lr, batch_size):
    for param, grad in zip(params, grads):
        param.assign_sub(lr*grad/batch_size)

In [5]:
lr = 0.03
num_epochs = 5
batch_size = 10

w, b = init_params()
for epoch in range(num_epochs):
    for X, y in data_iter(features, labels, batch_size):
        with tf.GradientTape() as tape:
            # Do forward prop & Compute loss
            loss = squared_loss(lin_reg(X, w, b), y)

        # Compute Gradient
        dw, db = tape.gradient(loss, [w, b])

        # Update w, b
        sgd([w, b], [dw, db], lr, batch_size)

    training_loss = squared_loss(lin_reg(features, w, b), labels)
    print(f"epoch : {epoch}, training_loss : {tf.reduce_mean(training_loss)}")

epoch : 0, training_loss : 0.03946446254849434
epoch : 1, training_loss : 0.0001582117547513917
epoch : 2, training_loss : 5.046507067163475e-05
epoch : 3, training_loss : 4.991786045138724e-05
epoch : 4, training_loss : 4.995410199626349e-05


In [6]:
print(w, b)

<tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[ 3.000012 ],
       [-2.1999488]], dtype=float32)> <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([4.2003527], dtype=float32)>


## Tensorflow Concise Implementation

In [7]:
import tensorflow as tf
import numpy as np

In [8]:
## Generate Data

### Init Variables
true_w = tf.constant([3, -2.2])
true_b = 4.2
num_samples = 1000

### Generate Samples
features = tf.random.normal((num_samples, true_w.shape[0]), dtype=tf.float32)
labels = tf.matmul(features, tf.reshape(true_w, (-1, 1))) + true_b
labels += tf.random.normal(shape=labels.shape, stddev=0.01)
labels = tf.reshape(labels, (-1, 1))

In [9]:
## Read Batches of data

def load_data(data, batch_size, is_train = False):
    """ Construct Tensorflow data iterator
    is_train : to shuffle data """
    dataset = tf.data.Dataset.from_tensor_slices(data)
    if is_train:
        dataset = dataset.shuffle(buffer_size = 1000)
    dataset = dataset.batch(batch_size)
    return dataset

# batch_size = 10
# data_iter = load_data((features, labels), batch_size)
# # next(iter(data_iter))

In [10]:
## Initialize parameters

initializer = tf.initializers.RandomNormal(stddev = 0.01)
lin_reg_tf = tf.keras.Sequential()
lin_reg_tf.add(tf.keras.layers.Dense(1, kernel_initializer = initializer))

In [11]:
## Set Optimizer and loss
trainer = tf.keras.optimizers.SGD(learning_rate = 0.05)
squared_loss = tf.keras.losses.MeanSquaredError()

In [12]:
num_epochs = 5
batch_size = 100
data_iter = load_data((features, labels), batch_size)
for epoch in range(num_epochs):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            # Do forward prop & Compute loss
            loss = squared_loss(lin_reg_tf(X, training = True), y)
        
        # Calculate Gradient
        grads = tape.gradient(loss, lin_reg_tf.trainable_variables)
    
        # Update weights
        trainer.apply_gradients(zip(grads, lin_reg_tf.trainable_variables))
    
    # Compute epoch loss
    training_loss = squared_loss(lin_reg_tf(features), labels)

    print(f"epoch : {epoch}, training_loss : {tf.reduce_mean(training_loss)}")


epoch : 0, training_loss : 3.8380420207977295
epoch : 1, training_loss : 0.47112494707107544
epoch : 2, training_loss : 0.05801168829202652
epoch : 3, training_loss : 0.007222828920930624
epoch : 4, training_loss : 0.0009677072521299124


In [13]:
lin_reg_tf.get_weights()

[array([[ 2.9864178],
        [-2.1858115]], dtype=float32),
 array([4.177178], dtype=float32)]

## Gaussian Distribution and Linear Regression

* $ Gaussian \ Distribution : \frac{1}{\sqrt{2\pi\sigma^2}}* exp[-\frac{(x - \mu)^2}{2\sigma^2}] $


* In Lin Reg, the observations are assumed to arise from noisy observations -> $\epsilon$ ~ $N(0, \sigma^2)$


* Likelihood of seeing y for given x:

 $ P(y|x) = \frac{1}{\sqrt{2\pi\sigma^2}}* exp[-\frac{(y - w^Tx - b)^2}{2\sigma^2}] $
 
 
* By principle of max likelihood, best value of w, b for entire dataset which maximize the likelihood:

    $ P(y|X) = \prod p(y|x) $
 
 
* The abv eq has prod of exp -> diff to work with -> take log and minimize with - sign

     $ -log(P(y|X)) = \sum \frac{1}{2} log (2\pi\sigma^2) + \frac{1}{2\sigma^2} (y - w^Tx - b)^2 $
 
 
* The second term is MSE and need to be minimized
 